In [0]:
%pip install -q faker mimesis

In [0]:
from databricks.sdk import WorkspaceClient
from datetime import datetime

ws = WorkspaceClient()

dbutils.widgets.dropdown("num_rows", defaultValue="1000", choices=["1000", "10000", "100000", "1000000"])
catalogs = [x.full_name for x in list(ws.catalogs.list())]
dbutils.widgets.dropdown("catalog", defaultValue=catalogs[0], choices=catalogs)
schemas = [x.name for x in list(ws.schemas.list(catalog_name=dbutils.widgets.get("catalog")))]
dbutils.widgets.dropdown("schema", defaultValue=schemas[0], choices=schemas)
num_rows = int(dbutils.widgets.get("num_rows"))
dbutils.widgets.text("table_name", defaultValue=f"fake_pii_data_{int(datetime.now().timestamp())}")

In [0]:
from faker import Faker
from mimesis import Person, Address, Payment, Internet
from mimesis.locales import Locale
import random
from pyspark.sql.types import *
import pandas as pd

fake = Faker('en_US')
person = Person(Locale.EN)
address = Address()
payment = Payment()
internet = Internet()

schema = StructType([
    StructField("name", StringType(), False),
    StructField("email", StringType(), False),
    StructField("passport", StringType(), False),
    StructField("phone_number", StringType(), False),
    StructField("ipv4", StringType(), False),
    StructField("ipv6", StringType(), False),
    StructField("address", StringType(), False),
    StructField("location", StringType(), False),
    StructField("ssn", StringType(), False),
    StructField("itin", StringType(), False),
    StructField("bank_number", StringType(), False),
    StructField("iban", StringType(), False),
    StructField("credit_card_number", StringType(), False)
    ])

def generate_data(pdf):

    pdf["name"] = random.choice([fake.name(), person.full_name()])
    pdf["email"] = random.choice([fake.email(), fake.company_email(), fake.free_email(), person.email()])
    pdf["passport"] = random.choice([fake.passport_number(), fake.passport_full()])
    pdf["phone_number"] = random.choice([fake.phone_number(), fake.basic_phone_number(), person.telephone()])
    pdf["ipv4"] = random.choice([fake.ipv4(), fake.ipv4_private(), fake.ipv4_public(), internet.ip_v4(), internet.ip_v4_with_port()]) 
    pdf["ipv6"] = random.choice([fake.ipv6(), internet.ip_v6()]) 
    pdf["address"] = random.choice([fake.address(), address.address()])
    pdf["location"] = random.choice([fake.city(), fake.country(), fake.state(), fake.street_address(), address.country()])
    pdf["ssn"] = fake.ssn()
    pdf["itin"] = fake.itin()
    pdf["bank_number"] = fake.bban() # Todo!
    pdf["iban"] = fake.iban()
    pdf["credit_card_number"] = random.choice([fake.credit_card_number(), payment.credit_card_number()])
    # us_driver_license = Todo!
    return pdf

def generate_fake_data(pdf: pd.DataFrame) -> pd.DataFrame:

    return pdf.apply(generate_data, axis=1).drop(["id", "partition_id"], axis=1)

In [0]:
from pyspark.sql.functions import spark_partition_id

df = (spark.range(0, num_rows).withColumn("partition_id", spark_partition_id())
      .groupBy("partition_id")
      .applyInPandas(generate_fake_data, schema))
display(df)

In [0]:
df.write.mode("overwrite").saveAsTable(f"{dbutils.widgets.get('catalog')}.{dbutils.widgets.get('schema')}.{dbutils.widgets.get('table_name')}")